In [1]:
import pandas as pd
from sqlalchemy import create_engine, text
import cred_pg as c

In [2]:
engine = create_engine(
    f'postgresql://{c.pg_userid}:{c.pg_password}@{c.pg_host}/{c.pg_db}', 
    connect_args = {'options': '-c search_path=usozmed,public', 'keepalives_idle': 120},
    pool_size=1, 
    max_overflow=0,
    execution_options={ 'isolation_level': 'AUTOCOMMIT' }
)

In [3]:
def exe(sql):
    with engine.connect() as con:
        return pd.read_sql_query(text(sql), con)

## Q1

In [4]:
sql = """
select count(*)
from 
  country co
  join city ci on ci.partofcountryid = co.id
  join person p on p.locationcityid = ci.id
  join forum_hasmember_person fmbr on fmbr.personid = p.id
  join forum f on f.id = fmbr.forumid
  join message msgp on msgp.containerforumid = f.id
  join message msgc on msgc.parentmessageid = msgp.id
  join message_hastag_tag mtag on mtag.messageid = msgc.id
  join tag t on t.id = mtag.tagid
  join tagclass tc on tc.id = t.typetagclassid;
"""
exe(sql) # 155.615.588, 10-11 sec

,count
0,155615588


## Q2

In [14]:
sql = """
select count(*)
from 
  person p1
  join person_knows_person knows on knows.person1id = p1.id
  join person p2 on p2.id = knows.person2id
  join message msgc on msgc.creatorpersonid = p1.id
  join message msgp on msgp.creatorpersonid = p2.id 
and msgp.parentmessageid is null and msgc.parentmessageid = msgp.id;
"""
exe(sql) # 296644, 0.5s

,count
0,296644


## Q3

In [16]:
sql = """
select count(*)
from 
  country co
  join city as ci_a on ci_a.partofcountryid  = co.id
  join city as ci_b on ci_b.partofcountryid  = co.id
  join city as ci_c on ci_c.partofcountryid  = co.id
  join person as p1 on p1.locationcityid  = ci_a.id
  join person as p2 on p2.locationcityid  = ci_b.id
  join person as p3 on p3.locationcityid  = ci_c.id
  join person_knows_person as knows_a on knows_a.person1id = p1.id and knows_a.person2id = p2.id
  join person_knows_person as knows_b on knows_b.person1id = p1.id and knows_b.person2id = p3.id
  join person_knows_person as knows_c on knows_c.person1id = p2.id and knows_c.person2id = p3.id;
"""
exe(sql) # 71354, 3.5s

,count
0,71354


## Q4

In [12]:
sql = f"""
select count(*)
from
  tag t
  join message_hastag_tag mtag on mtag.tagid = t.id
  join message msg on msg.id = mtag.messageid
  join message msgc on msgc.parentmessageid = msg.id
  join person_likes_message plikes on plikes.messageid = msg.id
  join person p_liker on p_liker.id = plikes.personid 
"""
exe(sql) # 9853666, 1.3s

,count
0,9853666


## Q5
41.6M 3,37 / 0,48

In [13]:
sql = """
select count(*)
from
  tag t1
  join message_hastag_tag as mtag1 on mtag1.tagid = t1.id
  join message comment on comment.id = mht1.messageid
  join message on message.id = comment.parentmessageid
  join message_hastag_tag mht2 on mht2.messageid = message.id
  where mht1.tagid != mht2.tagid;
"""
exe(sql) # 9679562, 1.7s

,count
0,9679562


## Q6
6.3B 6,50 / 4,76

In [14]:
sql = f"""
SELECT count(*)
FROM Person_knows_Person pkp1
JOIN Person_knows_Person pkp2 ON pkp1.Person2Id = pkp2.Person1Id AND pkp1.Person1Id != pkp2.Person2Id
JOIN Person_hasInterest_Tag ON pkp2.Person2Id = Person_hasInterest_Tag.personid;
"""
exe(sql) # 2.2s

,count
0,176011014


## Q7
85.2M

In [ ]:
sql = f"""
SELECT count(*)
FROM Message_hasTag_Tag mht
JOIN Message m on m.Id = mht.MessageId
JOIN Person p ON p.id = m.creatorpersonid
LEFT JOIN Comment_replyOf_Message ON Comment_replyOf_Message.ParentMessageId = Message_hasTag_Tag.MessageId
LEFT JOIN Person_likes_Message ON Person_likes_Message.MessageId = Message_hasTag_Tag.MessageId;
"""
exe(sql)

## Q8
20.8M 

In [ ]:
sql = f"""
SELECT count(*)
FROM Message_hasTag_Tag
JOIN Comment_replyOf_Message
ON Message_hasTag_Tag.MessageId = Comment_replyOf_Message.
ParentMessageId
JOIN Comment_hasTag_Tag AS cht1
ON Comment_replyOf_Message.CommentId = cht1.id
LEFT JOIN Comment_hasTag_Tag AS cht2
ON Message_hasTag_Tag.hasTag_Tag = cht2.hasTag_Tag
AND Comment_replyOf_Message.CommentId = cht2.id
WHERE Message_hasTag_Tag.hasTag_Tag != cht1.hasTag_Tag
AND cht2.hasTag_Tag IS NULL;
"""
exe(sql)

## Q9
6.0B 11,86 / 12,83

In [20]:
sql = f"""
SELECT count(*)
FROM Person_knows_Person pkp1
JOIN Person_knows_Person pkp2 ON pkp1.Person2Id = pkp2.Person1Id AND pkp1.Person1Id != pkp2.Person2Id
JOIN Person_hasInterest_Tag ON pkp2.Person2Id = Person_hasInterest_Tag.PersonId
LEFT JOIN Person_knows_Person pkp3 ON pkp3.Person1Id = pkp1.Person1Id AND pkp3.Person2Id = pkp2.Person2Id
WHERE pkp3.Person1Id IS NULL;
"""
exe(sql) # 8.8s

,count
0,169140023
